In [1]:
library(lme4)
library(mbest)
library(arm)
library(dplyr)

Loading required package: Matrix
Loading required package: nlme

Attaching package: ‘nlme’

The following object is masked from ‘package:lme4’:

    lmList

Loading required package: MASS

arm (Version 1.9-3, built: 2016-11-21)

Working directory is /Users/tsweetser/sf/diamond/examples/logistic


Attaching package: ‘dplyr’

The following object is masked from ‘package:MASS’:

    select

The following object is masked from ‘package:nlme’:

    collapse

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
df = read.csv("soybean_small.csv")

In [3]:
head(df)

date,plant_stand,precip,temp,hail,crop_hist,area_damaged,severity,seed_tmt,germination,⋯,sclerotia,fruit_pods,fruit.spots,seed,mold_growth,seed_discolor,seed_size,shriveling,roots,NA.
4,0,2,1,1,1,0,1,0,2,⋯,0,0,4,0,0,0,0,0,0,D1
5,0,2,1,0,3,1,1,1,2,⋯,0,0,4,0,0,0,0,0,0,D1
3,0,2,1,0,2,0,2,1,1,⋯,0,0,4,0,0,0,0,0,0,D1
6,0,2,1,0,1,1,1,0,0,⋯,0,0,4,0,0,0,0,0,0,D1
4,0,2,1,0,3,0,2,0,2,⋯,0,0,4,0,0,0,0,0,0,D1
5,0,2,1,0,2,0,1,1,0,⋯,0,0,4,0,0,0,0,0,0,D1


In [4]:
summary(dplyr::select(df, fruiting_bodies, precip, leaves))

 fruiting_bodies      precip          leaves      
 Min.   :0.0000   Min.   :0.000   Min.   :0.0000  
 1st Qu.:0.0000   1st Qu.:1.000   1st Qu.:1.0000  
 Median :0.0000   Median :2.000   Median :1.0000  
 Mean   :0.2128   Mean   :1.489   Mean   :0.8085  
 3rd Qu.:0.0000   3rd Qu.:2.000   3rd Qu.:1.0000  
 Max.   :1.0000   Max.   :2.000   Max.   :1.0000  

In [5]:
model <- mhglm(fruiting_bodies ~ 1 + precip + (1 + precip | leaves),
                 data = df,
                 family = binomial)

In [6]:
summary(model)


Call:
mhglm(formula = fruiting_bodies ~ 1 + precip + (1 + precip | 
    leaves), family = binomial, data = df)

Random effects:
 Groups Name        Variance Std.Dev Corr
 leaves (Intercept) 0        0           
        precip      0        0       NaN 

Fixed effects:
            Estimate Std. Error z value Pr(>|z|)  
(Intercept)   -3.885      1.911  -2.033   0.0420 *
precip         1.651      1.003   1.647   0.0996 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [7]:
ranef(model)$leaves

,(Intercept),precip
0,0,0
1,0,0


In [8]:
get_priors <- function(model, group, lme4=TRUE) {
  priors <- VarCorr(model)
  if (lme4) {
    priors <- as.data.frame(priors)
    priors <- mutate(priors, var1 = ifelse(var1 == "(Intercept)", "intercept", var1))
    names(priors) <- tolower(names(priors))    
  } else {
    # for now, only take the diagonal
    priors <- data.frame(
        grp   = rep(group, ncol(priors[[group]])),
        var1  = tolower(colnames(priors[[group]])),
        var2  = rep(NA, ncol(priors[[group]])),
        vcov  = diag(priors[[group]]),
        sdcor = sqrt(diag(priors[[group]])),
        # this needs to be here due to bad R quirks
        stringsAsFactors = FALSE
    )
    priors$var1[priors$var1 == '(intercept)'] <- 'intercept'
    rownames(priors) <- NULL
  }
  return(priors)
}

In [9]:
priors = get_priors(model, group="leaves", lme4=FALSE)

In [10]:
colnames(priors)[1] <- 'group'
priors

group,var1,var2,vcov,sdcor
leaves,intercept,NA,0,0
leaves,precip,NA,0,0


In [11]:
write.csv(priors, "./soybean_small_priors.csv", row.names=FALSE)